### 학습/테스트 데이터 셋 분리 – train_test_split()

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target

테스트 데이터를 사용하지 않고, 학습 데이터로 예측하면 무슨 문제가 발생하는지 확인

In [3]:
# 학습 데이터로 학습 수행
dt_clf.fit(train_data, train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [4]:
# 학습 데이터로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

# 정확도가 100% 나옴 -> 뭔가 이상함
# 100%가 나온 이유는 학습한 데이터로 예측을 했기 때문
# 즉, 모의고사를 통해 답을 알고 있는 상태에서 같은 문제로 시험을 봤기 때문

예측 정확도: 1.0


학습 데이터로 학습하고, 테스트 데이터로 예측 수행

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [6]:
dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

In [7]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=121)                                                                                     

In [8]:
dt_clf.fit(X_train, y_train)  # 학습 수행
pred = dt_clf.predict(X_test) # 예측 수행
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))  # 평가 수행

# 테스트 데이터로 예측한 결과 95% 나옴
# 붓꽃 데이터는 150개로 데이터 양이 많지 않아 테스트 데이터는 45개 밖에 안됨 -> 알고리즘 성능 파악하기 힘듬

예측 정확도: 0.9556


### 교차 검증
- 학습과 테스트 데이터로 분리하는 것은 과적합의 문제를 가져올 수 있음
  - 모델이 학습 데이터에 과도하게 최적화되어 예측을 다른 데이터로 하면 예측 성능이 떨어지게 됨
  - 고정된 학습과 테스트 데이터로 평가를 하면 테스트 데이터에만 최적의 성능이 나오는 편향된 모델을 만들게 됨
- 이런 문제를 해결하기 위한 방법이 교차 검증
  - 즉, 수능(테스트 데이터)을 보기 전, 모의고사(교차검증의 학습 데이터와 검증 데이터)를 여러번 학습 함
  - 데이터 편중을 막기위함
  - 학습 데이터, 검증 데이터, 테스트 데이터

#### K 폴드 교차 검증
- 가장 보편적으로 사용되는 교차 검증 
- K개의 데이터 폴드 세트를 만들어 K번 만큼 각 폴드 세트에 학습과 검증 평가를 반복으로 수행
- K번째까지 학습과 검증을 수행
- K개의 예측 평가를 구했으면 이를 평균내서 K폴드 결과로 반영
- 사이킷런에서 KFold 클래스 제공

KFold 클래스 이용해 붓꽃 데이터를 교차 검증하고 예측 정확도 알아보기

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [10]:
# 붓꽃 데이터와 DecisionTreeClassifier클래스의 객체 생성
iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

In [11]:
kfold = KFold(n_splits=5)  # 5개의 폴드로 분리하는 KFold 객체 생성 -> 5번의 학습과 검증을 함
cv_accuracy = []           # 5번 학습과 검증 결과로 나온 예측 성능을 리스트로 담음
print('붓꽃 데이터 세트 크기:',features.shape[0])

# 전체 붓꽃 데이터는 모두 150개
# 5개로 분리했으므로 학습 데이터는 4/5인 120개, 검증 데이터는 1/5인 30개로 분리됨

붓꽃 데이터 세트 크기: 150


KFold객체는 split하면 학습용, 검증용 데이터로 분할할 수 있는 인덱스를 반환함<br>
학습과 검증 데이터를 반복해 예측 정확도를 측정<br>
split이 어떤 값을 반환하는지 보기 위해 검증 데이터 인덱스를 추출해 봄

In [12]:
n_iter = 0

for train_index, test_index  in kfold.split(features): # KFold객체에 split(features)하면 학습, 검증 데이터로 나눠짐
                                        # 150개의 피쳐를 학습 데이터를 가지는 위치 인덱스, 검증 데이터를 가지는 위치 인덱스로 반환
                                        # 이 작업을 5번 수행해야하므로 for문 사용
    
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]  # 학습 데이터 위치 인덱스로 학습 데이터 추출
    y_train, y_test = label[train_index], label[test_index]        # 검증 데이터 위치 인덱스로 검증 데이터 추출
    
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)     # 학습 수행
    pred = dt_clf.predict(X_test)     # 예측 수행
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(y_test,pred), 4)  # 정확도 측정
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))       # 학습 데이터와 검증 데이터가 120:30으로 만들어지는지 확인하기 위함
  
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))  # 검증 인덱스에 test_index가 어떤 값인지 확인해 봄
    
    cv_accuracy.append(accuracy)  # 리스트에 정확도 값을 넣음
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))  # 예측 성능 평균을 냄

# 5번 교차 검증 결과 평균 검증 정확도는 0.9
# 교차 검증 시마다 검증 세트의 인덱스가 달라지는 것을 확인할 수 있음
# 첫번째 교차 검증은 0~29, 두번째는 30~59 등등 각각 30개씩 검증 인덱스 생성함, 이를 기반으로 검증 세트 추출


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


#### Stratified K 폴드
- 불균형한 분포도를 가진 레이블 데이터 집합을 위한 방식
  -  특정 레이블 값이 많거나 적어서 값의 분포가 한쪽으로 치우치는 경우
- 원본 데이터와 유사한 레이블 값의 분포를 학습/테스트 세트에서도 유지해야 함
  - 원본 데이터의 레이블 분포를 먼저 고려한 뒤, 이 분포와 동일하게 학습과 검증 데이터를 분배해 줌

먼저 K폴드가 가진 문제점을 확인해 봄

In [13]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# 레이블 값의 분포도 확인
iris_df['label']=iris.target
iris_df['label'].value_counts()

# 레이블 값은 0,1,2 모두 50개로 동일함 

2    50
1    50
0    50
Name: label, dtype: int64

문제점을 발생시키기 위해 3개의 폴드 세트를 KFold로 생성하고, 각 교차 검증시마다 생성되는 학습/검증 레이블 분포도 확인해 봄

In [14]:
kfold = KFold(n_splits=3)

n_iter =0
for train_index, test_index  in kfold.split(iris_df): # kfold.split(X): 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 데이터 행 인덱스 반환 
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]  # train_index, test_index라는 위치 인덱스를 받아 iloc로 데이터 가져옴
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    
    # 학습 레이블과 검증 레이블 분포도를 확인해 봄
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

# 교차검증: 1을 보면 학습 레이블은 1,2뿐이라 0은 학습되지 않음
# 검증 레이블은 0뿐이라 학습 모델은 0을 예측하지 못함
# 즉, 이렇게 분리하면 검증 예측 정확도는 0이 나옴

# KFold로 분할된 레이블 데이터가 전체 레이블 분포도를 반영하지 못함

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


문제점을 해결하기 위해 stratifiedkfold 사용

In [15]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
                                       # 피처데이터, 레이블데이터
                                       # 결정값을 넣어야 이걸 보고 분포를 고루 쪼갤 수 있음
            
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    
# 결과를 보면 학습 레이블과 검증 레이블 분포가 동일하게 할당됨

# 교차 검증: 1의 학습 레이블에 0,1,2가 33으로 동일하게 할당됨
# 검증 레이블도 0,1,2가 17로 동일하게 할당됨
# 이렇게 분할이 되어야 레이블 0,1,2 모두 학습할 수 있음

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 2    34
1    34
0    34
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    16
1    16
0    16
Name: label, dtype: int64


stratifiedKFold 이용해 붓꽃 데이터 교차 검증

In [16]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label): # 같은 코드에 label만 추가됨
    
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 

# 3개의 stratifiedk폴드로 교차 검증한 결과 평균 검증 정확도가 96.04%나옴
# 즉, 분류에서의 교차 검증은 k폴드가 아닌 stratifiedk폴드로 해야함
# 하지만 회귀에서는 stratifiedk폴드가 지원되지 않음 -> 회귀의 결정값은 레이블이 아닌 숫자이므로 분포를 정하는게 의미가 없음


#1 교차 검증 정확도 :0.9804, 학습 데이터 크기: 99, 검증 데이터 크기: 51
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]

#2 교차 검증 정확도 :0.9216, 학습 데이터 크기: 99, 검증 데이터 크기: 51
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133]

#3 교차 검증 정확도 :0.9792, 학습 데이터 크기: 102, 검증 데이터 크기: 48
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.9804 0.9216 0.9792]
## 평균 검증 정확도: 0.9604


#### 교차 검증을 보다 간편하게 cross_val_score()
- 학습, 예측, 평가를 한번에 수행해 줌

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)  

# 데이터 입력
data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy) , 교차 검증 폴드 수는 3개 
scores = cross_val_score(dt_clf , data , label , scoring='accuracy',cv=3)  # 분류, 피처, 레이블, 평가지표,교차 검증 폴드 수
                                     # cv는 3이므로 교차 검증세트는 3개이며, 정확도를 가지고 트리를 학습하고 검증으로 평가하여 정확도 추출
    
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

# 앞의 stratified k 폴드와 결과가 동일하게 나옴

# 각각의 교차검증 세트별로 수행한 결과값들이 나옴
# 결과값들을 평균적으로 나타냄

교차 검증별 정확도: [0.9804 0.9216 0.9792]
평균 검증 정확도: 0.9604


#### GridSearchCV - 교차 검증과 최적 하이퍼 파리미터 튜닝을 한 번에
- 하이퍼 파라미터를 조정해 알고리즘 예측 성능을 개선할 수 있음
- 사이킷런은 gridsearchCV로 분류나 회귀에 사용되는 최적의 하이퍼 파라미터를 도출하게 해줌

결정 트리 알고리즘의 최고 성능 하이퍼 파라미터 조합을 찾기위해 파라미터 집합 만들고 이를 순차적으로 적용하면서 최적화를 수행해 봄

In [18]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩
iris = load_iris()

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)

# 트리 객체 생성
dtree = DecisionTreeClassifier()

# parameter들을 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]} # 딕셔너리 형태로 파라미터 이름과 값을 리스트 형태로 넣어줌

# 하이퍼 파라미터는 12,13,22,23,32,33으로 총 6번에 걸쳐 바뀜

In [19]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)  # refit=True가 default: 최적의 하이퍼 파라미터로 재학습 

# 학습 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 
grid_dtree.fit(X_train, y_train)   # fit하는 순간 파라미터를 순차적으로 3세트에 맞춰 수행함 (18번 수행)

# GridSearchCV 결과를 이해하기 쉽게 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)        # cv_results_에 결과값이 딕셔너리로 저장됨
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [21]:
# 가장 좋은 결과로 나온 파라미터로 학습시킨 결과
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

# refit=true로 설정된 gridsearchcv객체를 fit하면 best_estimator_속성은 이미 학습이 된 estimator를 가지고 있음
# gridsearchcv에서 바로 predict 하지않고, 이미 학습된 estimator로 데이터를 분리하여 예측하고 성능을 평가할 수 있음
estimator = grid_dtree.best_estimator_
        
# 예측, 평가 수행
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

# 별도의 테스트 데이터에서 수행한 결과

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750
테스트 데이터 세트 정확도: 0.9667


일반적으로 학습 데이터를 gridsearchcv이용해 최적 하이퍼 파라미터 튜닝을 한뒤, 별도의 테스트 데이터에서 평가하는 것이 일반적